In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("../input/titanic/train.csv")
test_data = pd.read_csv("../input/titanic/test.csv")

In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
train_data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [5]:
train_data["Age"].fillna(train_data["Age"].mean(), inplace=True)
test_data["Age"].fillna(test_data["Age"].mean(), inplace=True)

In [6]:
train_data = train_data.drop(columns=["Name"])
train_data["Sex"] = train_data["Sex"].map({"male" : 1, "female" : 0})
train_data["Embarked"].fillna(train_data["Embarked"].mode()[0], inplace=True)


test_data = test_data.drop(columns=["Name"])
test_data["Sex"] = test_data["Sex"].map({"male" : 1, "female" : 0})
test_data["Embarked"].fillna(test_data["Embarked"].mode()[0], inplace=True)

train_data.Embarked.isna().sum()


0

In [7]:
train_data = train_data.join(pd.get_dummies(train_data["Embarked"], prefix="Embarked"))
train_data.drop(["Embarked", "Ticket"], axis=1, inplace=True)

test_data = test_data.join(pd.get_dummies(test_data["Embarked"], prefix="Embarked"))
test_data.drop(["Embarked", "Ticket"], axis=1, inplace=True)

In [8]:
train_data["cabin_letters"] = train_data.Cabin.apply(lambda x: str(x)[0])
test_data["cabin_letters"] = test_data.Cabin.apply(lambda x: str(x)[0])


In [9]:
train_data = train_data.join(pd.get_dummies(train_data['cabin_letters'], prefix="cabin_letters"))
train_data.drop(["Cabin", "cabin_letters"], axis=1, inplace=True)

test_data = test_data.join(pd.get_dummies(test_data['cabin_letters'], prefix="cabin_letters"))
test_data.drop(["Cabin", "cabin_letters"], axis=1, inplace=True)




In [10]:
print(train_data.isna().sum())
print(test_data.isna().sum())

PassengerId        0
Survived           0
Pclass             0
Sex                0
Age                0
SibSp              0
Parch              0
Fare               0
Embarked_C         0
Embarked_Q         0
Embarked_S         0
cabin_letters_A    0
cabin_letters_B    0
cabin_letters_C    0
cabin_letters_D    0
cabin_letters_E    0
cabin_letters_F    0
cabin_letters_G    0
cabin_letters_T    0
cabin_letters_n    0
dtype: int64
PassengerId        0
Pclass             0
Sex                0
Age                0
SibSp              0
Parch              0
Fare               1
Embarked_C         0
Embarked_Q         0
Embarked_S         0
cabin_letters_A    0
cabin_letters_B    0
cabin_letters_C    0
cabin_letters_D    0
cabin_letters_E    0
cabin_letters_F    0
cabin_letters_G    0
cabin_letters_n    0
dtype: int64


In [11]:
test_data['Fare'].fillna(test_data['Fare'].mean(), inplace=True)

Scaling and standardization of data

In [12]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

features = [ 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Fare', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'cabin_letters_A',
       'cabin_letters_B', 'cabin_letters_C', 'cabin_letters_D',
       'cabin_letters_E', 'cabin_letters_F', 'cabin_letters_G'] 

train_y = train_data["Survived"]
train_data.drop("Survived", axis=1, inplace=True)

train_data_scaled = ss.fit_transform(train_data[features])
test_data_scaled = ss.transform(test_data[features])

In [13]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

xgb = XGBClassifier(random_state=1)

xgb.fit(train_data_scaled, train_y)
predictions = xgb.predict(test_data_scaled)

output= pd.DataFrame({'PassengerId' : test_data.PassengerId, 'Survived' : predictions})
output.to_csv('submission.csv', index= False)